In [ ]:
!pip install transformers
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!apt-get install git-lfs
!git lfs install
!git clone https://huggingface.co/taeminlee/kogpt2

Detected operating system as Ubuntu/jammy.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Detected apt version as 2.4.12
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... Packagecloud gpg key imported to /etc/apt/keyrings/github_git-lfs-archive-keyring.gpg
done.
Running apt-get update... done.

The repository is setup! You can now install packages.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following packages will be upgraded:
  git-lfs
1 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Need to get 7,420 kB of archives.
After this operation, 6,051 kB of additional disk space will be used.
Get:1 https://packagecloud.io/github/git-lfs/ubuntu jammy/main amd64 git-lfs amd64 3.5.1 [7,420 kB]
Fetched 7,420 kB in 0s (24.2 MB/s)
(Reading database ... 121757 files and directories c

In [ ]:
import torch
from tokenizers import SentencePieceBPETokenizer
from transformers import GPT2Config, GPT2LMHeadModel

tokenizer = SentencePieceBPETokenizer("/content/kogpt2/vocab.json", "/content/kogpt2/merges.txt")

config = GPT2Config(vocab_size=50000)
config.pad_token_id = tokenizer.token_to_id('<pad>')
model = GPT2LMHeadModel(config)

model_dir = '/content/kogpt2/pytorch_model.bin'

model.load_state_dict(torch.load(model_dir, map_location=torch.device('cpu')), strict=False)
model.to('cpu')

tokenizer.add_special_tokens(["<s>", "</s>"])

2

GPT가 생성한 문장을 받아오기 위한 함수를 구현해보겠습니다.

- 토크나이저를 활용해 토큰화합니다.
- sampling 기법을 활용해 토큰들을 선택하고, 문장을 생성합니다.
- top-k와 top-p를 각각 50, 0.95로 설정합니다.
- 반환하는 문장은 1개로 설정합니다. (num_return_sequences)
- 문장의 끝을 알리는 스페셜 토큰을 설정해줍니다. (eos_token_id)
- 동일한 토큰이 2회이상 반복되지 못하게 설정합니다. (no_repeat_ngram_size)

생성 전략을 위와 같이 설정하고, 생성된 문장을 반환하는 함수입니다.

In [ ]:
def get_gpt_output(input_sent):
    input_ids = torch.tensor(tokenizer.encode(input_sent, add_special_tokens=True).ids).unsqueeze(0).to('cpu')
    sample_outputs = model.generate(
        input_ids,
        do_sample=True,
        max_length=512,
        top_k=50,
        top_p=0.95,
        num_return_sequences=1,
        eos_token_id=tokenizer.token_to_id("</s>"),
        no_repeat_ngram_size=2,
        early_stopping=True
    )
    generated_sequence = sample_outputs[0].tolist()
    return tokenizer.decode(generated_sequence, skip_special_tokens=True)

In [ ]:
get_gpt_output("<s>철수 : 영희야 안녕!</s><s>영희 : 어! 철수야! 오랜만이다!</s><s>철수 : 그러게~ 잘 지냈어?</s><s>영희 : ")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:453: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


'철수 : 영희야 안녕! 영희 : 어! 철수야! 오랜만이다! 철수 : 그러게~ 잘 지냈어? 영희 : !!!!!'

In [ ]:
get_gpt_output("<s>본문 : 아.. 기분 진짜 짜증나네ㅡㅡ</s><s>감정 : 분노</s><s>본문 : 와!! 진짜 너무 좋아!!</s><s>감정 : ")

'본문 : 아.. 기분 진짜 짜증나네 감정 : 분노 본문 : 와!! 진짜 너무 좋아!! 감정 : ^^* [연재] 3년, 3개월, 연재 : 317편* 로맨스편(37) III * 끝났습니다.'

In [ ]:
get_gpt_output("<s>질문 : 코로나 바이러스에 걸리면 어떻게 되나요?</s>\
<s>답 : COVID-19 환자는 일반적으로 감염 후 평균 5 ~ 6 일 (평균 잠복기 5 ~ 6 일, 범위 1 ~ 14 일)에 경미한 호흡기 증상 및 발열을 포함한 징후와 증상을 나타냅니다. COVID-19 바이러스에 감염된 대부분의 사람들은 경미한 질병을 앓고 회복됩니다.</s>\
<s>질문 : 코로나 바이러스 질병의 첫 증상은 무엇입니까?</s>\
<s>답 : 이 바이러스는 경미한 질병에서 폐렴에 이르기까지 다양한 증상을 유발할 수 있습니다. 질병의 증상은 발열, 기침, 인후통 및 두통입니다. 심한 경우 호흡 곤란과 사망이 발생할 수 있습니다.</s>\
<s>질문 : 딸기 식물의 수명주기는 무엇입니까?</s>\
<s>답 : 딸기의 생애는 새로운 식물의 설립으로 시작하여 2 ~ 3 년 후 절정에 이르렀다가 절정에 이어 2 ~ 3 년에 노화와 죽음을 향해 진행됩니다. 이상적인 조건에서 딸기 식물은 5-6 년까지 살 수 있습니다.</s>\
<s>질문 : 파이썬 메서드의 self 매개 변수의 목적은 무엇입니까?</s>\
<s>답 : self 매개 변수는 클래스의 현재 인스턴스에 대한 참조이며 클래스에 속한 변수에 액세스하는 데 사용됩니다.</s>\
<s>질문 : 뇌의 어떤 부분이 말을 제어합니까?</s>\
<s>답 : 언어 우세 반구의 왼쪽 전두엽 (브로카 영역)에있는 뇌의 분리 된 부분에 대한 손상은 자발적 언어 및 운동 언어 제어 사용에 상당한 영향을 미치는 것으로 나타났습니다.</s>\
<s>질문 : 인공지능의 미래에 대해 어떻게 생각하십니까?</s>\
<s>답 : ")

'질문 : 코로나 바이러스에 걸리면 어떻게 되나요? 답 : COVID-19 환자는 일반적으로 감염 후 평균 5 ~ 6 일 (평균 잠복기 5 ~ 6 일, 범위 1 ~ 14 일)에 경미한 호흡기 증상 및 발열을 포함한 징후와 증상을 나타냅니다. COVID-19 바이러스에 감염된 대부분의 사람들은 경미한 질병을 앓고 회복됩니다. 질문 : 코로나 바이러스 질병의 첫 증상은 무엇입니까? 답 : 이 바이러스는 경미한 질병에서 폐렴에 이르기까지 다양한 증상을 유발할 수 있습니다. 질병의 증상은 발열, 기침, 인후통 및 두통입니다. 심한 경우 호흡 곤란과 사망이 발생할 수 있습니다. 질문 : 딸기 식물의 수명주기는 무엇입니까? 답 : 딸기의 생애는 새로운 식물의 설립으로 시작하여 2 ~ 3 년 후 절정에 이르렀다가 절정에 이어 2 ~ 3 년에 노화와 죽음을 향해 진행됩니다. 이상적인 조건에서 딸기 식물은 5-6 년까지 살 수 있습니다. 질문 : 파이썬 메서드의 self 매개 변수의 목적은 무엇입니까? 답 : self 매개 변수는 클래스의 현재 인스턴스에 대한 참조이며 클래스에 속한 변수에 액세스하는 데 사용됩니다. 질문 : 뇌의 어떤 부분이 말을 제어합니까? 답 : 언어 우세 반구의 왼쪽 전두엽 (브로카 영역)에있는 뇌의 분리 된 부분에 대한 손상은 자발적 언어 및 운동 언어 제어 사용에 상당한 영향을 미치는 것으로 나타났습니다. 질문 : 인공지능의 미래에 대해 어떻게 생각하십니까? 답 : rain planner를 활용해 뇌파의 파파린(정전기) 및 후두파파파를 제어하는 방법을 개발할 수 있다고 보고되었습니다. partner pensive는 이러한 기술을 이용하여 로봇에 의해 작동되는 언어와 행동을 조절합니다. ------------------------------ --- rank forest는 자동적인 파파를 생성하는 파파로, 두뇌가 작동하며, 이를 통해 두뇌는 현재 자신의 음성이나 문자, 컴퓨터나 기타 첨단 기술을 이용할 수 없습니다. https://www.facebook

In [ ]:
get_gpt_output("<s>한국어: 그 도로는 강과 평행으로 뻗어 있다.</s>\
<s>English: The road runs parallel to the river.</s>\
<s>한국어: 그 평행선들은 분기하는 것처럼 보인다.</s>\
<s>English: The parallel lines appear to diverge.</s>\
<s>한국어: 그 도로와 운하는 서로 평행하다.</s>\
<s>English: The road and the canal are parallel to each other.</s>\
<s>한국어: 평행한 은하계라는 개념은 이해하기가 힘들다.</s>\
<s>English: The idea of a parallel universe is hard to grasp.</s>\
<s>한국어: 이러한 전통은 우리 문화에서는 그에 상응하는 것이 없다.</s>\
<s>English: This tradition has no parallel in our culture.</s>\
<s>한국어: 이것은 현대에 들어서는 그 유례를 찾기 힘든 업적이다.</s>\
<s>English: This is an achievement without parallel in modern times.</s>\
<s>한국어: 그들의 경험과 우리 경험 사이에서 유사점을 찾는 것이 가능하다.</s>\
<s>English: It is possible to draw a parallel between their experience and ours.</s>\
<s>한국어: 그 새 학위 과정과 기존의 수료 과정이 동시에 운영될 수도 있을 것이다.</s>\
<s>English: The new degree and the existing certificate courses would run in parallel.</s>\
<s>한국어: 이순신은 조선 중기의 무신이다.</s>\
<s>Englisth: ")

'한국어: 그 도로는 강과 평행으로 뻗어 있다. English: The road runs parallel to the river. 한국어: 그 평행선들은 분기하는 것처럼 보인다. English: The parallel lines appear to diverge. 한국어: 그 도로와 운하는 서로 평행하다. English: The road and the canal are parallel to each other. 한국어: 평행한 은하계라는 개념은 이해하기가 힘들다. English: The idea of a parallel universe is hard to grasp. 한국어: 이러한 전통은 우리 문화에서는 그에 상응하는 것이 없다. English: This tradition has no parallel in our culture. 한국어: 이것은 현대에 들어서는 그 유례를 찾기 힘든 업적이다. English: This is an achievement without parallel in modern times. 한국어: 그들의 경험과 우리 경험 사이에서 유사점을 찾는 것이 가능하다. English: It is possible to draw a parallel between their experience and ours. 한국어: 그 새 학위 과정과 기존의 수료 과정이 동시에 운영될 수도 있을 것이다. English: The new degree and the existing certificate courses would run in parallel. 한국어: 이순신은 조선 중기의 무신이다. Englisth: all specific research, looks a magnificer. in the mention. is easy with edito-indic. for in daured acrossment. of the indietigentes as a rather in age desers delming adapething, and when change to try in

In [ ]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!apt-get install git-lfs

!git lfs install
!git clone https://huggingface.co/taeminlee/kogpt2
!pip install transformers

Detected operating system as Ubuntu/jammy.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Detected apt version as 2.4.12
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... Packagecloud gpg key imported to /etc/apt/keyrings/github_git-lfs-archive-keyring.gpg
done.
Running apt-get update... done.

The repository is setup! You can now install packages.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.5.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Git LFS initialized.
fatal: destination path 'kogpt2' already exists and is not an empty directory.


In [ ]:
import torch
from tokenizers import SentencePieceBPETokenizer
from transformers import GPT2Config, GPT2LMHeadModel

tokenizer = SentencePieceBPETokenizer("/content/kogpt2/vocab.json", "/content/kogpt2/merges.txt")

config = GPT2Config(vocab_size=50000)
model = GPT2LMHeadModel(config)

model_dir = '/content/kogpt2/pytorch_model.bin'

model.load_state_dict(torch.load(model_dir, map_location='cpu'), strict=False)
model.to('cpu')

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50000, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50000, bias=False)
)

In [ ]:
!git clone https://github.com/songys/Chatbot_data.git

import pandas as pd
data = pd.read_csv('/content/Chatbot_data/ChatbotData.csv')

data.head(10)

Cloning into 'Chatbot_data'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 69 (delta 18), reused 4 (delta 4), pack-reused 48
Receiving objects: 100% (69/69), 398.65 KiB | 1.58 MiB/s, done.
Resolving deltas: 100% (35/35), done.


,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
5,SD카드 망가졌어,다시 새로 사는 게 마음 편해요.,0
6,SD카드 안돼,다시 새로 사는 게 마음 편해요.,0
7,SNS 맞팔 왜 안하지ㅠㅠ,잘 모르고 있을 수도 있어요.,0
8,SNS 시간낭비인 거 아는데 매일 하는 중,시간을 정하고 해보세요.,0
9,SNS 시간낭비인데 자꾸 보게됨,시간을 정하고 해보세요.,0


In [ ]:
added_special_token_num = tokenizer.add_special_tokens(['<s>', '</s>'])

pad_id = tokenizer.token_to_id("<pad>")
tokenizer.enable_padding(pad_id=pad_id, pad_token="<pad>")
tokenizer.enable_truncation(max_length=128)


In [ ]:
class ChatDataset(torch.utils.data.Dataset):
    def __init__(self, tokenizer, file_path):
        self.data = []
        self.file_path = file_path
        self.tokenizer = tokenizer

    def load_data(self):
        raw_data = pd.read_csv(self.file_path)
        train_data = '<s>'+raw_data['Q']+'</s>'+'<s>'+raw_data['A']+'</s>'
        #<s>안녕하세요</s><s> -> 네, 안녕하세요</s>
        tokenized_train_data = tokenizer.encode_batch(train_data)
        for single_data in tokenized_train_data:
            self.data.append(torch.tensor(single_data.ids).unsqueeze(0))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        item = self.data[index]
        return item

train_dataset = ChatDataset(tokenizer=tokenizer, file_path='/content/Chatbot_data/ChatbotData.csv')
train_dataset.load_data()

In [ ]:
from torch.utils.data import DataLoader

data_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

In [ ]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=1e-4, correct_bias=True)

epochs = 3

avg_loss = (0.0, 0.0)
for epoch in range(epochs):
    count=0
    for data in data_loader:
        optimizer.zero_grad()
        data = data.transpose(1,0)
        data = data.to('cpu')
        model = model.to('cpu')

        outputs = model(data, labels=data)
        loss, logits = outputs[:2]
        loss = loss.to('cpu')
        loss.backward()
        avg_loss = (avg_loss[0] * 0.99 + loss, avg_loss[1] * 0.99 + 1.0)
        optimizer.step()
        if count % 200 == 0:
            print('epoch no.{0}  train ({1}/{2})  loss = {3:.5f}  avg_loss = {4:.5f}' . format(epoch, count, len(data_loader), loss, avg_loss[0] / avg_loss[1]))
        count += 1

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


epoch no.0  train (0/2956)  loss = 2.62272  avg_loss = 2.62272


In [ ]:
torch.save(model.state_dict(), 'chitchat_model.bin')